In [ ]:
!pip install cv2
!pip install pytesseract
!pip install numpy

In [2]:
from PIL import Image
import cv2
import numpy as np
import os
import matplotlib as plt
import math
import scipy.ndimage

In [24]:
def remove_noise(image_path, save_path):
    # Open the image using Pillow
    image = Image.open(image_path)

    # Convert the image to grayscale
    gray_image = image.convert('L')

    # Convert PIL image to numpy array
    np_image = np.array(gray_image)

    # Apply binary thresholding
    _, binary_image = cv2.threshold(np_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Apply morphological operations
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, kernel, iterations=1)  # Increase iterations for more noise removal

    # Apply Gaussian blur
    denoised_image = cv2.GaussianBlur(opening, (3, 3), 0)

    # Save the denoised image
    denoised_image_pil = Image.fromarray(denoised_image)
    denoised_image_pil.save(save_path)


In [4]:
# Example usage
folder = 'four_cap_36'
image_file = 'ZZW3-18954.png'
path = os.path.join(folder, image_file)
remove_noise(path, 'denoised_captcha.png')

In [13]:
def crop_minAreaRect(img, rect):
# https://stackoverflow.com/questions/37177811/crop-rectangle-returned-by-minarearect-opencv-python
    # rotate img
    center = rect[0]
    size = rect[1]
    print("size[0]: " + str(int(size[0])) + ", size[1]: " + str(int(size[1])))
    angle = rect[2]
    print("angle: " + str(angle))
    rows,cols = img.shape[0], img.shape[1]
    M = cv2.getRotationMatrix2D((cols/2,rows/2),angle,1)
    img_rot = cv2.warpAffine(img,M,(cols,rows))

    # rotate bounding box
    rect0 = (rect[0], rect[1], angle) 
    box = cv2.boxPoints(rect0)
    pts = np.int0(cv2.transform(np.array([box]), M))[0]    
    pts[pts < 0] = 0

    # crop
    img_crop = img_rot[pts[1][1]:pts[0][1], pts[1][0]:pts[2][0]]

    w, h = img_crop.shape[0], img_crop.shape[1]
    print("w_cropped: " + str(w) + ", h_cropped: " + str(h))
    return img_crop

def sort_contours(cnts, method="left-to-right"):
# from https://pyimagesearch.com/2015/04/20/sorting-contours-using-python-and-opencv/    
    reverse = False
    i = 0
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),   key=lambda b:b[1][i], reverse=reverse))
    return (cnts, boundingBoxes)    
    
im_name = 'denoised_captcha.png'
im = cv2.imread(im_name)
im_copy = im.copy()

imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(imgray, 127, 255, 0)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#cv2.drawContours(im_copy, contours, -1, (0,255,0), 2)
#cv2.imshow("contours", im_copy)

print("num contours: " + str(len(contours)))
i = 0

sorted_cnts, bounding_boxes = sort_contours(contours, method="left-to-right")

for cnt in sorted_cnts:
  size = cv2.contourArea(cnt)
  x,y,w,h = cv2.boundingRect(cnt)
  rect = cv2.minAreaRect(cnt)
#  print(str(rect))
#  if rect[1][0] > 0 and rect[1][1]>0:
  im_cropped = crop_minAreaRect(im, rect)
    
  h,w = im_cropped.shape[0], im_cropped.shape[1]
  if w > h:
    im_cropped = cv2.rotate(im_cropped, cv2.ROTATE_90_CLOCKWISE)  
    
  print("w: " + str(w) + ", h: " + str(h))
  

num contours: 5
size[0]: 59, size[1]: 19
angle: 86.56636810302734
w_cropped: 0, h_cropped: 59
w: 59, h: 0
size[0]: 49, size[1]: 15
angle: 86.90594482421875
w_cropped: 7, h_cropped: 50
w: 50, h: 7
size[0]: 3, size[1]: 2
angle: 90.0
w_cropped: 2, h_cropped: 3
w: 3, h: 2
size[0]: 53, size[1]: 17
angle: 77.27564239501953
w_cropped: 18, h_cropped: 53
w: 53, h: 18
size[0]: 53, size[1]: 17
angle: 78.36637115478516
w_cropped: 9, h_cropped: 54
w: 54, h: 9


C:\Users\luizf\AppData\Local\Temp\ipykernel_53024\3189924673.py:16: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  pts = np.int0(cv2.transform(np.array([box]), M))[0]


In [18]:
import cv2
import os

def save_contours_as_images(image_path, output_directory):
    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Threshold the image to obtain binary image
    _, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    for i, contour in enumerate(contours):
        # Get bounding box for each contour
        x, y, w, h = cv2.boundingRect(contour)

        # Check if contour is too small (possibly noise)
        if w > 5 and h > 5:
            # Crop the character region from the image
            character = image[y:y+h, x:x+w]

            # Save the cropped character as a separate image
            character_filename = os.path.join(output_directory, f'character_{i+1}.png')
            cv2.imwrite(character_filename, character)

# Example usage:
image_path = 'denoised_captcha.png'
output_directory = 'cropped_characters'
save_contours_as_images(image_path, output_directory)


In [25]:
# Example usage
# folder = 'four_cap_36'
# image_file = '0CMI-33396.png'
# path = os.path.join(folder, image_file)
# remove_noise(path, 'denoised_captcha.png')

image_path = 'denoised_captcha.png'
output_directory = 'cropped_characters'
save_contours_as_images(image_path, output_directory)